In [ ]:
import sys, os, glob
print sys.executable

import numpy as np
import pyuvdata.utils as uvutils
import matplotlib.gridspec as gridspec

from pyuvdata import UVData
from pyuvdata.data import DATA_PATH
from astropy.time import Time

import matplotlib
matplotlib.rcParams['figure.figsize'] = (16, 9)
from matplotlib.pyplot import *
%matplotlib qt5

## Read in uvfits file  
Choose data with complete filepath via uvfits_file variable

In [ ]:
uvfiles = os.path.join(DATA_PATH,'day2_TDEM0003_10s_norx_1src_1spw.uvfits')

miriadfiles = None

# Read in uvfits data
UV = UVData()
if not uvfiles is None:
    UV.read_uvfits(uvfiles)
elif not miriadfiles is None:
    UV.read_miriad(miriadfiles)

# Check for the number of spectral windows
# This notebook only supports one spectral window
if UV.Nspws > 1:
    print 'Plotting currently only supported for one spectral window.'

# Earth rotation radians/s
EARTH_OMEGA = 7.29e-5 

## Set data and plotting options
Descriptions of parameters available at bottom of notebook

In [ ]:
""" Data options """

# ant_str = ant_str

antenna_nums = None

antenna_names = None

ant_pairs_nums = None

frequencies = None

freq_chans = None

times = None

polarizations = None

blt_inds = None



""" Plotting Options """

plot_type = 'abs'

no_white_space = True

dmin,dmax = None,None

force_clim = False

cmap = get_cmap('gnuplot')

log_scale = True

time_axis = 'index'

chan_axis = 'index'

fontsize = 16

decimate = None 

decimate_offset = None 

unmask = False 

In [ ]:
""" PLOTTING """

if fignum_exists(1):
    clf()

# Quality of life function
def get_ints_from_str(x):
    return map(int, x.split(','))


# Check for a supported plot_type option
if not plot_type in ['abs', 'real', 'imag', 'phase']:
    print 'Please choose a supported plot type'


# Perform selection
if not ant_str is None:
    UV_selected = UV.select(frequencies = frequencies,
                        freq_chans = freq_chans,
                        times = times,
                        blt_inds = blt_inds,
                        inplace = False)
    antpair_pols = UV.parse_ants(ant_str=ant_str, for_select=False)
    if ant_str == 'all':
        for pair in UV_selected.get_antpairs():
            antpair_pols[str(pair).strip('()')] = uvutils.polstr2num(UV_selected.get_pols())
else:
    UV_selected = UV.select(antenna_nums = antenna_nums,
                            antenna_names = antenna_names,
                            ant_pairs_nums = ant_pairs_nums,
                            frequencies = frequencies,
                            freq_chans = freq_chans,
                            times = times,
                            polarizations = polarizations,
                            blt_inds = blt_inds,
                            inplace = False)
    antpair_pols = {}
    for pair in UV_selected.get_antpairs():
        antpair_pols[str(pair).strip('()')] = uvutils.polstr2num(UV_selected.get_pols())


# Masking/unmasking
if unmask:
    data_array = np.ma.masked_array(UV_selected.data_array,
                                    mask=False,
                                    fill_value=1e20)
else:
    data_array = np.ma.masked_array(UV_selected.data_array,
                                    mask=UV_selected.flag_array,
                                    fill_value=1e20)
    cmap.set_over(alpha=0.0)

    
# Decimate data
if not decimate is None:
    if not decimate_offset is None:
        data_array = data_array[decimate_offset::decimate,:,:,:]
    else:
        data_array = data_array[::decimate,:,:,:]

# If plotting phase, force linear scale
if plot_type == 'phase':
    log_scale = False

# If plotting a single baseline, set no_white_space to False for proper formatting
if len(antpair_pols.keys()) == 1:
    no_white_space = False

# If no_white_space, automatically set force_clim=True
if no_white_space:
    force_clim = True


# Assign range for colorbar range if dmin and dmax not specified
if dmin is None and dmax is None:
    if log_scale:
        if plot_type == 'abs':
            dmin = np.ma.min(np.log10(np.abs(data_array)))
            dmax = np.ma.max(np.log10(np.abs(data_array)))
        elif plot_type == 'real':
            dmin = np.ma.min(np.log10(np.real(data_array)))
            dmax = np.ma.max(np.log10(np.real(data_array)))
        elif plot_type == 'imag':
            dmin = np.ma.min(np.log10(np.imag(data_array)))
            dmax = np.ma.max(np.log10(np.imag(data_array)))
        else:
            print 'Cannot have log_scale with plot_type= \'phase\''
    else:
        if plot_type == 'abs':
            dmin = np.ma.min(np.abs(data_array))
            dmax = np.ma.max(np.abs(data_array))
        elif plot_type == 'real':
            dmin = np.ma.min(np.real(data_array))
            dmax = np.ma.max(np.real(data_array))
        elif plot_type == 'imag':
            dmin = np.ma.min(np.imag(data_array))
            dmax = np.ma.max(np.imag(data_array))
        else:
            dmin = -np.pi
            dmax = np.pi 


# Check for infinities in dmin, dmax
if dmin == -np.inf:
    if log_scale:
        dmin = -10
    else:
        dmin = 1e-10
elif dmax == np.inf:
    if log_scale:
        dmax = 10
    else:
        dmax = 1e10


# Determine necessary number of rows/cols for subplots
bl_keys = antpair_pols.keys()
nplots = 0
for key in bl_keys:
    nplots += len(antpair_pols[key])
nrows = int(np.sqrt(nplots))
ncols = int(np.ceil(float(nplots) / nrows))
nempty = nrows*ncols - nplots
       

# Create figure and gridspec object

fig = figure(1,facecolor='w')
gs = gridspec.GridSpec(nrows,ncols)


if no_white_space:
    # Add master labels for all x and y axes
    master_ax = fig.add_subplot(gs[:,:])
    
    if chan_axis == 'index':
        master_ax.set_xlabel('Frequency [channels]', size=fontsize, labelpad=25)
    elif chan_axis == 'physical':
        master_ax.set_xlabel('Frequency [MHz]', size=fontsize, labelpad=25)
    
    if time_axis == 'index':
        master_ax.set_ylabel('Time [integrations]', size=fontsize, labelpad=35)
    if time_axis == 'lst':
        master_ax.set_ylabel('LST [hours]', size=fontsize, labelpad=35)
    
    master_ax.set_xticks([])
    master_ax.set_yticks([])
    master_ax.spines['top'].set_visible(False)
    master_ax.spines['right'].set_visible(False)
    master_ax.spines['bottom'].set_visible(False)
    master_ax.spines['left'].set_visible(False)
    if not uvfiles is None:
        ttl = master_ax.set_title(uvfiles)
    elif not miriadfiles is None:
        ttl = master_ax.set_title(miriadfiles)
    ttl.set_position([0.5, 1.05])

# Parameters for annotating subplots with baseline information
x_lbl_loc = 0.95 # x location of antenna pair label if no_white_space
y_lbl_loc = 0.09 # y location of antenna pair label if no_white_space
plot_ind = [0,0] # [row,col] in gridspec layout
bls_counter = 0 # counter for baseline in each subplot
pol_counter = 0 # counter for polarization for each baseline

# Do the plotting
while plot_ind[0] < nrows and bls_counter < len(bl_keys):
    while plot_ind[1] < ncols and bls_counter < len(bl_keys):
        
        # Get indices for given baseline
        bl_key = bl_keys[bls_counter]
        antpair = get_ints_from_str(bl_key)
        pol = antpair_pols[bl_key][pol_counter]
        bl_inds, bl2_inds, pol_inds = UV_selected._key2inds((antpair[0], antpair[1], pol))
        
        ax = fig.add_subplot(gs[plot_ind[0],plot_ind[1]])

        if plot_type == 'abs':
            if log_scale:
                im = ax.imshow(np.ma.log10(np.abs(data_array[bl_inds,0,:,pol_inds])),
                               origin='upper', aspect='auto', interpolation='none',
                               cmap=cmap)
            else:
                im = ax.imshow(np.abs(data_array[bl_inds,0,:,pol_inds]),
                               origin='upper', aspect='auto', interpolation='none',
                               cmap=cmap)
        elif plot_type == 'real':
            if log_scale:
                im = ax.imshow(np.ma.log10(np.real(data_array[bl_inds,0,:,pol_inds])),
                               origin='upper', aspect='auto', interpolation='none',
                               cmap=cmap)
            else:
                im = ax.imshow(np.real(data_array[bl_inds,0,:,pol_inds]),
                               origin='upper', aspect='auto', interpolation='none',
                               cmap=cmap)
        elif plot_type == 'imag':
            if log_scale:
                im = ax.imshow(np.ma.log10(np.imag(data_array[bl_inds,0,:,pol_inds])),
                               origin='upper', aspect='auto', interpolation='none',
                               cmap=cmap)
            else:
                im = ax.imshow(np.imag(data_array[bl_inds,0,:,pol_inds]),
                               origin='upper', aspect='auto', interpolation='none',
                               cmap=cmap)
        else:
            im = ax.imshow(np.angle(data_array[bl_inds,0,:,pol_inds]),
                           origin='upper', aspect='auto', interpolation='none',
                           cmap=cmap)
        
        if force_clim:
            # Force plots to same clim
            im.set_clim(dmin,dmax)

        # Make label for subplot
        pol_str = uvutils.polnum2str(pol)
        ax_label = str(antpair[0])+pol_str[0]+','+str(antpair[1])+pol_str[1]
        
        # Set Image extent
        if time_axis == 'lst':
            elapsed = (UV_selected.lst_array[bl_inds].max()
                       - UV_selected.lst_array[bl_inds].min())/(3600.*EARTH_OMEGA)
        if chan_axis == 'physical':
            plotfreqs = [np.round(UV_selected.freq_array[0,0]/1.e6,0), 
                         np.round(UV_selected.freq_array[0,-1]/1.e6, 0)]
        
        if chan_axis == 'index' and time_axis == 'index':
            ax_label_x = x_lbl_loc*UV_selected.Nfreqs
            ax_label_y = y_lbl_loc*UV_selected.Ntimes
        elif chan_axis == 'index' and time_axis == 'lst':
            extent = [0, UV_selected.Nfreqs, elapsed, 0]
            ax_label_x = x_lbl_loc*UV_selected.Nfreqs
            ax_label_y = y_lbl_loc*elapsed
            im.set_extent(extent)
        elif chan_axis == 'physical' and time_axis == 'index':
            extent = [plotfreqs[0], plotfreqs[1], UV_selected.data_array[bl_inds].shape[0], 0]
            ax_label_x = x_lbl_loc*plotfreqs[1]
            ax_label_y = y_lbl_loc*UV_selected.Ntimes
            im.set_extent(extent)
        elif chan_axis == 'physical' and time_axis == 'lst':
            extent = [plotfreqs[0], plotfreqs[1], elapsed, 0]
            ax_label_x = x_lbl_loc*plotfreqs[1]
            ax_label_y = y_lbl_loc*elapsed
            im.set_extent(extent)
        
        if no_white_space:
            # remove axis labels from interior plots
            if nrows*ncols == nplots:
                if not plot_ind[0] == nrows-1:
                    ax.set_xticks([])
            else:
                if plot_ind[0] < nrows-2:
                    ax.set_xticks([])
                elif plot_ind[0] == nrows-2 and plot_ind[1] <= ncols-nempty-1:
                    ax.set_xticks([])

            if not plot_ind[1] == 0:
                ax.set_yticks([])

            # Add antenna numbers to subplot
            ax.text(ax_label_x, ax_label_y,
                    ax_label, ha='right', size=fontsize)  
        else:
            # Add title and axis labels to each subplot
            ax.set_title(ax_label,size=fontsize)
            
            # Set x-axis labels
            if chan_axis == 'index':
                ax.set_xlabel('Frequency [channels]', size=fontsize)
            elif chan_axis == 'physical':
                ax.set_xlabel('Frequency [MHz]', size=fontsize)
                
            # Set y-axis labels
            if time_axis == 'index':
                ax.set_ylabel('Time [integrations]', size=fontsize)
            elif time_axis == 'lst':
                ax.set_ylabel('LST [hours]', size=fontsize)
                
            ax.tick_params(axis='both', labelsize=fontsize)

            # Add colorbars to subplots
            cb = fig.colorbar(im, ax=ax, format='%.1f')
            cb.ax.tick_params(labelsize=fontsize)
            
            if log_scale:
                cb.set_label('Log Visibilities', size=fontsize, labelpad=10)
            else:
                cb.set_label('Visibilities', size=fontsize, labelpad=10)
        
        

        # Update column counter
        plot_ind[1] += 1
        
        if pol_counter == len(antpair_pols[bl_key]) - 1:
            bls_counter += 1
            pol_counter = 0
        else:
            pol_counter += 1
        
    # Reset column counter and update row counter
    plot_ind[1] = 0
    plot_ind[0] += 1   


# Update plot dimensions to add master colorbar
gs.tight_layout(fig)

if no_white_space:
    # Append master colorbar
    gs.update(wspace=0.0, hspace=0.0, right=0.85)
    cbar_ax = fig.add_axes([0.875, 0.15, 0.02, 0.75])
    cb = fig.colorbar(im, cax=cbar_ax)
    cb.set_clim(dmin,dmax)
    
    if log_scale:
        cb.set_label('Log Visibilities', size=fontsize, labelpad=10)
    else:
        cb.set_label('Visibilities', size=fontsize, labelpad=10)
    
    cb.ax.tick_params(labelsize=fontsize)

___

# Options  
Defined user parameters can be used to modify the data plotted and the plot style.

### Data options

__ant_str__: String used for parsing antenna pair numbers and polarizations.  Basic options are 'auto', 'cross', or 'all'.  If 'all' is passed as ant_str, then ant_pairs_nums is returned as None so the data is not filtered based on antenna pair numbers in UV.select(...).  The ant_str can also contain:  

    1. Individual antenna number(s)
        - 1: returns all ant pairs containing ant number 1 (including the auto correlation)
        - 1,2: returns all antenna pairs containing antennas 1 and 2
    2. Individual baseline(s)
        - 1_2: returns only the antenna pair (1,2)
        - 1_2,1_3,1_10: returns ant pairs (1,2),(1,3),(1,10)
        - (1,2)_3: returns ant pairs (1,3),(2,3)  
        - 1_(2,3): returns ant pairs (1,2),(1,3)
    3. Antenna number(s) and polarization(s)
        - 1x: returns all ant pairs containing ant number 1 and polarizations xx and xy
        - 2x_3y: returns the antenna pair (2,3) and polarization xy
        - 1r_2l,1l_3l,1r_4r: returns ant pairs (1,2),(1,3),(1,4) and pols rr, ll, and rl  
        - (1x,2y)_(3x,4y): returns ant pairs (1,3),(1,4),(2,3),(2,4) and pols xx, yy, xy, and yx
        - 2l_3: returns ant pair (2,3) and polarizations ll and lr
        - 2r_3: returns ant pair (2,3) and polarizations rr and rl
        - 1l_3,2x_3: returns ant pairs (1,3),(2,3) and pols ll, lr, xx, and xy
        - 1_3l,2_3x: returns ant pairs (1,3),(2,3) and pols ll, rl, xx, and yx
    4. Stokes parameter(s): can be lower/uppercase
        - i,I: keeps only Stokes I
        - q,V: keeps both Stokes Q and V
    5. Minus sign(s): if a minus sign is present in front of an antenna number, it will not be kept in the data
        - 1_(-3,10): returns ant pair (1,10)
        - (1,2)_(-3,4): returns ant pairs (1,4),(2,4)
        - 1x_(-3y,10x): returns ant pair (1,10) and polarization xx

If ant_str is passed, ant_pairs_nums and polarizations will be overwritten.

__antenna_nums__: The antennas numbers to keep in the object (antenna positions and names for the removed antennas will be retained).  This cannot be provided if antenna_names is also provided.  

__antenna_names__: The antennas names to keep in the object (antenna positions and names for the removed antennas will be retained).  This cannot be provided if antenna_nums is also provided.  

__ant_pairs_nums__: Alternate way of passing a list of tuples of antenna numbers to UV.select(...).  Follows the formatting of the output of parse_ants given __ant_str__ above, i.e. ant_pairs_nums = [(9,10),(9,31)] will only keep baselines with the matching antenna pairs (9,10) and (9,31). Ordering of the numbers within the tuple does not matter.  

__frequencies__: The frequencies to keep in the object.  

__freq_chans__: The frequency channel numbers to keep in the object.  

__times__: The times to keep in the object.  Must be passed as a list of Julian Dates.

__polarizations__: The polarizations to keep in the object in integer format.  See uvutils.polstr2num for polarization formatting.  

__blt_inds__: The baseline-time indices to keep in the object. This is not commonly used.  

__inplace__: Determines whether a new object is created when UV.select(...) is run.  Must be declared as false for this notebook to work.  

----------

### Plotting options

__plot_type__: String that determines what aspect of the data is plotted.   Can be 'abs', 'real', 'imag', or 'phase'.  

__dmin__,__dmax__: Min and max value for plotting on the colorbar.  If left as None and no_white_space is False, each subplot will have its own min and max.  If left as None and no_white_space or force_clim is True, dmin and dmax will be set automatically and each subplot will be forced to the same min and max.  

__force_clim__: If true, all subplots are forced to the same min and max via dmin and dmax, respectively.  If dmin and dmax are left as None, the min and max values will be calculated automatically.  

__cmap__: Colormap for plotting.  Can be gist_earth, gist_heat, gist_stern, gist_yarg, hot, cool, gray, bone, spectral, copper, jet to name a few.  For a more complete list, see pylab.cm.datad.keys().

__log_scale__: If true, plot in log10.  If false, plot in linear units.

__time_axis__ = Can be 'index' or 'lst' to plot in units of integrations or hours, respectively.

__chan_axis__ = Can be 'index' or 'physical' to plot in units of channels or MHz, respectively.

__no_white_space__: If true, white space between subplots is removed, a master colorbar is used for all subplots, and force_clim is set to True so all subplots have the same min and max.  

__fontsize__: Sets the fontsize for each subplot's axes and colorbar labels, ticks, titles, etc.

___